In [ ]:
import geopandas as gpd
import os
import matplotlib.pyplot as plt


def path_list():
    path_municipios = os.getcwd() + 'municipios.shp'
    path_aeroportos = os.getcwd() + 'aeroportos.shp'
    path_rodovias = os.getcwd() + 'rodovias.shp'

    return [path_municipios, path_aeroportos, path_rodovias]


def open_shapefile():
    return [gpd.read_file(i) for i in path_list()]


def filter_municipios(state, shapes):
    return municipios[municipios['uf'] == state]


def airports(shape, municipio_filtrado):
    gdf = gpd.sjoin(shape, municipio_filtrado, op='within')
    total_airports = gdf.value_counts('TipoAero')
    total_pav = gdf.value_counts('pavimento')

    return gdf, total_airports, total_pav


def roads(shape, municipio_filtrado):
    gdf = gpd.overlay(shape, municipio_filtrado, how='intersection')
    gdf_proj = gdf.copy()
    gdf_proj['geometry'] = gdf_proj['geometry'].to_crs(epsg=5880)
    gdf_proj['distancia km'] = gdf_proj['geometry'].length / 1000
    distancias = gdf_proj.groupby('nm_tipo_tr').agg('sum')['distancia km']

    return gdf, distancias


def run(state):
    shapes = open_shapefile()
    municipio_filter = filter_municipios(state, shapes[0])
    gdf_airports, aeroportos, pavimento = airports(shapes[1], municipio_filter)
    gdf_roads, stats_distance = roads(shapes[2], municipio_filter)
    # print(aeroportos)
    # print(f'Estatísticas do Estado {state}')
    # print(pavimento)
    # print(stats_distance)
    # gdf_airports.plot()
    # gdf_roads.plot()

    return municipio_filter, gdf_airports, gdf_roads


run('RJ')

In [ ]:
path_rodovias = os.getcwd() + '/dados/rodovias.shp'
gpd.read_file(path_rodovias).crs  # m_tipo_tr